<a href="https://colab.research.google.com/github/Iispar/hlt-project/blob/main/course_project_2023_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT 2023 Project (Template)

- Student(s) Name(s): Iiro Partanen
- Date: -
- Chosen Corpus: emotion
- Contributions (if group project):

### Corpus information

- Description of the chosen corpus: 
Emotion is a dataset of English Twitter messages with six basic emotions: anger, fear, joy, love, sadness, and surprise. For more detailed information please refer to the paper.
- Paper(s) and other published materials related to the corpus: 
  - CARER: Contextualized Affect Representations for Emotion Recognition (Saravia et al., EMNLP 2018)
  - https://paperswithcode.com/dataset/emotion
- State-of-the-art performance (best published results) on this corpus: 95% f1

---

## 1. Setup

In [1]:
!pip3 install -q transformers datasets evaluate
!pip install trankit
!pip install optuna
import datasets
import sklearn.feature_extraction
import torch
import transformers
import numpy as np
import evaluate
import optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.or

---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [2]:
dset = datasets.load_dataset("emotion");
# check it works
print(dset);

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})


### 2.2. Preprocessing

In [3]:
# vectorizes one item
def vectorize_item(item):
    vectorized = vectorizer.transform([item["text"]]); # vectorize. Initialized below...
    non_zero_features = vectorized.nonzero()[1]; # get the nonzeros and we take only the columns of the nonzeros because our matrix is only one row.
    non_zero_features += 1; # index zero is for padding so let's avoid it by adding 1 to all.

    return {"input_ids":non_zero_features} 

In [4]:
dset.shuffle(); #shuffle dataset for safety

# vectorization.
vectorizer = sklearn.feature_extraction.text.CountVectorizer( # get the vectorizer.
    binary = True,
    max_features = 20000, # Selected 20k to start with.
    token_pattern = r"(?u)\b\w+\b" # Token pattern to include one char words.
    )

texts=[item["text"] for item in dset["train"]]; # get all texts from train
vectorizer.fit(texts); # fitting the vectorizer

# vectorize the whole dataset.
dset_tokenized = dset.map(vectorize_item,num_proc=4);
# check it works
print(dset_tokenized["train"][0]);

Map (num_proc=4):   0%|          | 0/16000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2000 [00:00<?, ? examples/s]

{'text': 'i didnt feel humiliated', 'label': 0, 'input_ids': [3620, 4931, 6438, 6495]}


In [5]:
# padding and batching
def collator(list_of_items):
    allLabels = [item["label"] for item in list_of_items]; # list of all labels.
    batch = {"labels": torch.tensor(allLabels)}; # create a tenstor for the item (batch)
    tensors = [];
    max_len = max(len(item["input_ids"]) for item in list_of_items); # longest example in the batch. Pad to here.
    for item in list_of_items:
        ids = torch.tensor(item["input_ids"]); # input ids to tensor
        padded = torch.nn.functional.pad(ids,(0,max_len-ids.shape[0])); # actual padding. Pads ids, from + to max with 0.
        tensors.append(padded); # appended ids to tensors
    batch["input_ids"] = torch.vstack(tensors); # stacks items as they are now same len. Now these are matrixes.
    return batch;

# check it works
batch=collator([dset_tokenized["train"][2],dset_tokenized["train"][7]])
print("Shape of labels:",batch["labels"].shape)
print("Shape of input_ids:",batch["input_ids"].shape)
print(batch["labels"])
print(batch["input_ids"])
     

Shape of labels: torch.Size([2])
Shape of input_ids: torch.Size([2, 13])
tensor([3, 4])
tensor([[    1,  4931,  5739,  5800,  6495,  6563,  8456, 10157, 13643, 15061,
             0,     0,     0],
        [    1,    34,   749,  2680,  4931,  6495,  7076,  7712,  8076,  9245,
          9325, 13339, 15116]])


---

## 3. Machine learning model

### 3.1. Model training

In [6]:
# Your code to train the machine learning model on the training set and evaluate the performance on the validation set here

# needs a config, we wil just pass it.
class MLPConfig(transformers.PretrainedConfig):
    pass;

# model
class MLP(transformers.PreTrainedModel):
      config_class = MLPConfig; # sets config
      #initilazition
      def __init__(self,config):
        super().__init__(config); # call the super with out config which is now pass..
        self.vocab_size = config.vocab_size; # embedding matrix row count
        # Build embedding of vocab size +1 x hidden size. +1 again because of padding.
        self.embedding = torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0);
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001); # initialization of the embedding values
        self.output = torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels); # output layer is the size of the labels x hidden size.

      # forward
      def forward(self,input_ids,labels=None):
        embedded = self.embedding(input_ids); # sum up all the embeddings
        embedded_summed = torch.sum(embedded,dim=1); # sum up across word dimension
        projected = torch.tanh(embedded_summed); # non-linearity
        logits = self.output(projected); # apply the outer layer
      

        ## calculates the loss
        if labels is not None:
            # calculates the loss.
            loss = torch.nn.CrossEntropyLoss();
            return (loss(logits,labels),logits);
        else:
            # if no labels, just return the logits
            return (logits,);
  
# config
mlp_config = MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=25,nlabels=6);

In [7]:
# training

# Set training arguments
trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy = "steps",
    logging_strategy = "steps",
    eval_steps = 500,
    logging_steps = 500,
    learning_rate = 1e-4, #learning rate of the gradient descent
    max_steps = 20000,
    load_best_model_at_end = True,
    per_device_train_batch_size = 128
)


# evaluating
accuracy = evaluate.load("accuracy");
def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels;
    predictions = np.argmax(outputs, axis=-1); #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels);

# actual training
mlp = MLP(mlp_config); # Make a the actual model  
early_stopping = transformers.EarlyStoppingCallback(5); # stop training if the eval loss is not getting better.

# params
trainer = transformers.Trainer(
    model = mlp,
    args = trainer_args,
    train_dataset = dset_tokenized["train"],
    eval_dataset = dset_tokenized["test"],
    compute_metrics = compute_accuracy,
    data_collator = collator,
    callbacks = [early_stopping]
)

# FINALLY!
trainer.train();


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.660800,1.536718,0.460500
1000,1.454200,1.400822,0.556500
1500,1.273700,1.243837,0.598000
2000,1.067300,1.080418,0.677500
2500,0.869100,0.931570,0.749000
3000,0.699700,0.804923,0.802500
3500,0.563500,0.701349,0.837500
4000,0.457200,0.618717,0.854000
4500,0.375400,0.553389,0.861000
5000,0.312300,0.502318,0.872000


### 3.2 Hyperparameter optimization

In [8]:
# Used optuna for optimization

def objective(trial):
    # Define the search space for hyperparameters
    learning_rate = trial.suggest_float("learning_rate", 1e-7, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 64, 128])
    epochs=trial.suggest_int('num_train_epochs', low = 2,high = 6),

    # params
    trainer_args = transformers.TrainingArguments(
        "mlp_checkpoints",
        evaluation_strategy = "steps",
        logging_strategy = "steps",
        eval_steps = 500,
        logging_steps = 500,
        learning_rate = learning_rate,
        max_steps = 30000,
        load_best_model_at_end = True,
        per_device_train_batch_size = batch_size,
        per_device_eval_batch_size = batch_size,
        num_train_epochs = epochs
    )

    # the model
    mlp = MLP(mlp_config)

    # train a model
    trainer = transformers.Trainer(
        model = mlp,
        args = trainer_args,
        train_dataset = dset_tokenized["train"],
        eval_dataset = dset_tokenized["validation"],
        compute_metrics = compute_accuracy,
        data_collator = collator,
        callbacks = [early_stopping]
    )

    trainer.train()
    eval_results = trainer.evaluate()
    return eval_results["eval_accuracy"] # return the best result.

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)


[I 2023-04-21 09:22:13,332] A new study created in memory with name: no-name-0848aaed-9778-441d-b737-43c671c0b26c
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.618100,1.514611,0.465000
1000,1.377400,1.334024,0.575500
1500,1.116100,1.113927,0.678500
2000,0.848800,0.909622,0.774000
2500,0.629700,0.745479,0.827500
3000,0.468600,0.621974,0.862500
3500,0.355300,0.533028,0.872500
4000,0.276000,0.469147,0.881000
4500,0.219800,0.423466,0.886500
5000,0.179000,0.390391,0.891500


[I 2023-04-21 09:23:51,957] Trial 0 finished with value: 0.8955 and parameters: {'learning_rate': 0.0001248560540517126, 'batch_size': 128, 'num_train_epochs': 2}. Best is trial 0 with value: 0.8955.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.658400,1.569147,0.352000
1000,1.493600,1.469309,0.488500
1500,1.353100,1.347704,0.580500
2000,1.185900,1.212260,0.635500
2500,1.014500,1.079239,0.706000
3000,0.855400,0.955875,0.760000
3500,0.715700,0.846444,0.797500
4000,0.597300,0.751807,0.829000
4500,0.499500,0.671926,0.850000
5000,0.419700,0.605132,0.861500


[I 2023-04-21 09:26:16,170] Trial 1 finished with value: 0.892 and parameters: {'learning_rate': 8.128924064678988e-05, 'batch_size': 128, 'num_train_epochs': 6}. Best is trial 0 with value: 0.8955.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


...

[I 2023-04-21 10:01:47,891] Trial 17 finished with value: 0.8925 and parameters: {'learning_rate': 0.0008932897948542663, 'batch_size': 8, 'num_train_epochs': 5}. Best is trial 9 with value: 0.9005.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.753400,1.742051,0.287500
1000,1.724700,1.714295,0.282500
1500,1.696100,1.687371,0.311500
2000,1.669000,1.662551,0.345000
2500,1.644500,1.640671,0.370500
3000,1.623100,1.621997,0.390000
3500,1.604600,1.606155,0.396500
4000,1.588700,1.592821,0.399500
4500,1.575100,1.581512,0.409500
5000,1.563400,1.571809,0.417000


[I 2023-04-21 10:04:57,154] Trial 18 finished with value: 0.564 and parameters: {'learning_rate': 9.03070482315174e-06, 'batch_size': 64, 'num_train_epochs': 3}. Best is trial 9 with value: 0.9005.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
500,1.433800,1.202036,0.647000
1000,0.774400,0.701572,0.848000
1500,0.359500,0.468643,0.885500
2000,0.197200,0.375987,0.886000
2500,0.127200,0.336301,0.888500
3000,0.090600,0.319945,0.890000
3500,0.068400,0.315203,0.889500
4000,0.053600,0.317301,0.889000
4500,0.043200,0.322506,0.890000
5000,0.035500,0.330279,0.891500


[I 2023-04-21 10:05:52,533] Trial 19 finished with value: 0.8895 and parameters: {'learning_rate': 0.0003153142148070851, 'batch_size': 128, 'num_train_epochs': 2}. Best is trial 9 with value: 0.9005.


### 3.3. Evaluation on test set

In [15]:
# Your code to evaluate the final model on the test set here
eval_results = trainer.evaluate(dset_tokenized["validation"])

print(eval_results)

{'eval_loss': 0.31981730461120605, 'eval_accuracy': 0.891, 'eval_runtime': 0.3156, 'eval_samples_per_second': 6336.458, 'eval_steps_per_second': 792.057, 'epoch': 112.0}


Having fun :)

In [10]:
# creates a dataset with one example
def create_example(texts):
  labels = [];
  for item in texts:
    labels.append(0);
  data = {
      'text': texts,
      'label': labels,
  }
  return datasets.Dataset.from_dict(data);

In [11]:
# Testing random sentences that could be tweeted to see what they give as the result :)
texts = ['wow last night was so cool would recommend to everyone',
         'jesus the concert last night was so bad',
         'cant stand to stay at the school with these annoying people',
         'just got an F on the test',
         'just got an A on the test',
         'wow i never tought this day would come',
         'I just feel so useless every day',
         'wow finally feel like people really like me',
         'Just came from a date and it was so fun. He seems so nice and I can\'t wait to call him again',
         'i have been having a really good time with my friends and i feel like they really do care about me'
         'wait what just happened. omd I can\'t believe my eyes',
         'i was really suprised with the results',
         'tomorrow i have to go meet with my ex boyfriend. I just can\'t stand him anymore hes such an asshole',
         'he just suddenly fell infront of me I couldn\'t believe it',
         'I love him so much that I can\'t wait to marry him'];
labels = ['sadness', 'joy', 'love', 'anger', 'fear', 'suprise'];
example = create_example(texts); #creates a dict with just the example
example_tokenized = example.map(vectorize_item, num_proc=4); # tokenize
for i in range(len(texts)):
  print(texts[i])
  prediction = trainer.predict(example_tokenized).predictions[i]; # predicts the label
  largest = max(prediction); # label with largest value
  labelOfLargest = labels[list(prediction).index(largest)]; # name of label
  print(f"{labelOfLargest} with confidence of: {largest}\n");

Map (num_proc=4):   0%|          | 0/14 [00:00<?, ? examples/s]

wow last night was so cool would recommend to everyone


joy with confidence of: 3.9080233573913574

jesus the concert last night was so bad


sadness with confidence of: 2.9793362617492676

cant stand to stay at the school with these annoying people


anger with confidence of: 0.5617761015892029

just got an F on the test


...

joy with confidence of: 2.96022891998291

i was really suprised with the results


joy with confidence of: 1.167738914489746

tomorrow i have to go meet with my ex boyfriend. I just can't stand him anymore hes such an asshole


fear with confidence of: 1.7576593160629272

he just suddenly fell infront of me I couldn't believe it


sadness with confidence of: 1.3029853105545044

I love him so much that I can't wait to marry him


joy with confidence of: 1.8817859888076782



---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

The corpus includes Twitter messages in english and they have been annotated with six basic emotions which are anger, fear, joy, love, sadness, and surprise.
By the paper the tweets have been selected with some hashtags and then annotated. The selected hastags can be seen from the paper.

If i understood correctly the tweets are gathered by hastags and these hastags define the emotion that is in the tweet for example #worried will be a tweet that displays fear and #fun will be joy and so on. So this has been annotated with distant supervision


### 4.2 Results

(Briefly summarize your results)

The percentage that I got was at best times in the test just over 90% and a little bit less in validation (ab 89-90%). 

Had some fun and tried the model on "tweets" that I came up with and saw some interesting results. As you can see it does hit the mark on some of the tests, but also alot of the results are so and so. For example "he just suddenly fell infront of me I couldn't believe it" and "wow finally feel like people really like me" are labeled sadness and joy. While yes I feel like these can be these labels, but I would label these as suprise and love. So while the results are fine sometimes they feel like they take the "easier" result.

After looking into the dataset I noticed that exactly these (love and suprise) are a bit rarer than the other categories. Especially suprise was also the lowest accuracy label in the paper. Also alot of the "tweets" that were regarding love and suprise were in my opinion a bit suprising and I would have maybe categorized them as something else. So this will effect the results when I use just tweets that I come up with my self and that aren't the same type as in the train data. I feel like this is also the 10% that is lacking in my model and it fails mostly with these labels.

I think joy and sadness are the two feelings that are predicted really well in the model as they seem to work with my own tweets also. This would be also logical as the feeling are a bit vague and fit a lot of different sentences. But they are also the two most represented feelings in the dataset.

Something else that is interesting in my opinion is the tweets "just got an A on the test" and "just got an F on the test" that were labeled as joy and fear. Was really happy to see this and proves that the machine actually learnt something by understanding that getting a F on a test is bad and getting an A on the test is good. :)

### 4.3 Relation to state of the art

(Compare your results to the state-of-the-art performance)

The state of the art models are in the 95% range and my model is just closing in on 90% depending on luck of the model at times.

Not bad in my opinion when taking into consideration how simple this model is and the models that reach 95% seem to be BERT based and a lot larger.

---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

(Briefly describe the process of annotation)

### 5.2 Conversion into dataset

In [12]:
# Your code to convert the annotations into a dataset here

### 5.3. Model evaluation on out-of-domain test set

In [13]:
# Your code to evaluate the model on the out-of-domain test set here

### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)

### 5.5. Annotated data

In [14]:
# Include your annotated out-of-domain data here